# Homework 2, Problem 6

## Generic Graph Class


In [1]:
from typing import List, Dict, Set, Tuple


class Graph:
    def __init__(self):
        self.adj: Dict[int, Set] = {}

    def add_vertex(self, v: int):
        self.adj[v - 1] = set()

    def add_edge(self, u: int, v: int):
        if u - 1 not in self.adj:
            self.add_vertex(u)
        if v - 1 not in self.adj:
            self.add_vertex(v)

        self.adj[u - 1].add(v - 1)
        self.adj[v - 1].add(u - 1)

    def remove_vertex(self, v: int):
        for nbr in self.adj[v - 1]:
            self.adj[nbr].remove(v - 1)
        del self.adj[v - 1]

    def N(self, v: int) -> Set[int]:
        to_show = {u + 1 for u in self.adj[v - 1]}
        return to_show

    def degree(self, v: int) -> int:
        return len(self.adj[v - 1])

    def n(self):
        return len(self.adj)

    def __repr__(self):
        to_show = {v + 1: {u + 1 for u in self.adj[v]} for v in self.adj}
        return to_show.__repr__()

    def __str__(self):
        to_show = {v + 1: {u + 1 for u in self.adj[v]} for v in self.adj}
        return to_show.__str__()

## Program 1


In [2]:
def prufer_to_graph(S: List[int]) -> Graph:
    """
    Running time: O(too long)
    Improvements: Exists but couldn't be bothered
    """
    n = len(S) + 2
    L = [i + 1 for i in range(n)]

    T = Graph()
    for i, s in enumerate(S):
        # gets minimum vertex not in S and connect it to the graph
        v = min([l for l in L if l not in S[i:]])
        T.add_edge(v, s)
        L.remove(v)

    # adds the last edge/vertex
    assert len(L) == 2
    u, v = [l for l in L]
    T.add_edge(u, v)

    return T


T = prufer_to_graph([1, 1, 3, 5, 5])
T

{2: {1}, 1: {2, 3, 4}, 4: {1}, 3: {1, 5}, 5: {3, 6, 7}, 6: {5}, 7: {5}}

## Program 2


In [3]:
def tree_to_prufer(T: Graph) -> List[int]:
    """
    Running time: O(too long) as well
    Improvements: Exists but couldn't be bothered too
    """
    tmp = Graph()
    tmp.adj = T.adj.copy()
    T = tmp

    S = []
    leaves = {v + 1 for v in T.adj if T.degree(v + 1) == 1}
    while T.n() > 2:
        # gets the smallest leaf
        l = min(leaves)
        leaves.remove(l)

        # add parent to sequence
        [parent] = [v for v in T.N(l)]
        S.append(parent)
        T.remove_vertex(l)

        # update collection of leaves
        if T.degree(parent) == 1:
            leaves.add(parent)

    del T
    return S


S = tree_to_prufer(T)
S

[1, 1, 3, 5, 5]

## Program 3


In [4]:
def gen_prufer_trees(n: int) -> List[Tuple[List[int], Graph]]:
    assert n >= 3
    perms = cartesian_product([[i + 1 for i in range(n)] for _ in range(n - 2)]).list()
    results = [(list(p), prufer_to_graph(p)) for p in perms]
    return results

### Verification

The verification will be done as follows. First, we will generate all the pairs of $(S, T)$ where
$S$ is the Prufer sequence and $T$ is the corresponding labelled tree. Then, we convert $T$ back to
a Prufer Sequence and assert that it is the same sequence as $S$. After that, we assert that for
every labelled tree $T$, $T$ is unique among all trees that show up in the returning results for
some given value $n$.

If all assertions pass, then the we would have that every tree that shows up is unique and is 1-to-1
with the sequence that shows up with it in a pair---verifying the code that we wrote.

In [5]:
def verify_results(results: List[Tuple[List[int], Graph]]):
    # converting T back to sequence and assert that it is the same as S
    for S, T in results:
        converted_pc = tree_to_prufer(T)
        assert all([x == y for x, y in zip(S, converted_pc)])

    # gets all adjacency map of the trees and asserts that it an adjacency map shows up only once
    all_trees = [tree.adj for _, tree in results]
    for _, T in results:
        assert all_trees.count(T.adj) == 1

    return "Passed!"

In [6]:
# just to show how Dict comparison works
A = {"a": 1, "b": 2}
B = {"a": 1, "b": 2}
C = {"a": 2, "b": 1}

print(A == B, A == C)

True False


##### $n=3$


In [7]:
n3_results = gen_prufer_trees(3)
n3_results

[([1], {2: {1}, 1: {2, 3}, 3: {1}}),
 ([2], {1: {2}, 2: {1, 3}, 3: {2}}),
 ([3], {1: {3}, 3: {1, 2}, 2: {3}})]

In [8]:
verify_results(n3_results)

'Passed!'

##### $n=4$


In [9]:
n4_results = gen_prufer_trees(4)
n4_results

[([1, 1], {2: {1}, 1: {2, 3, 4}, 3: {1}, 4: {1}}),
 ([1, 2], {3: {1}, 1: {2, 3}, 2: {1, 4}, 4: {2}}),
 ([1, 3], {2: {1}, 1: {2, 3}, 3: {1, 4}, 4: {3}}),
 ([1, 4], {2: {1}, 1: {2, 4}, 4: {1, 3}, 3: {4}}),
 ([2, 1], {3: {2}, 2: {1, 3}, 1: {2, 4}, 4: {1}}),
 ([2, 2], {1: {2}, 2: {1, 3, 4}, 3: {2}, 4: {2}}),
 ([2, 3], {1: {2}, 2: {1, 3}, 3: {2, 4}, 4: {3}}),
 ([2, 4], {1: {2}, 2: {1, 4}, 4: {2, 3}, 3: {4}}),
 ([3, 1], {2: {3}, 3: {1, 2}, 1: {3, 4}, 4: {1}}),
 ([3, 2], {1: {3}, 3: {1, 2}, 2: {3, 4}, 4: {2}}),
 ([3, 3], {1: {3}, 3: {1, 2, 4}, 2: {3}, 4: {3}}),
 ([3, 4], {1: {3}, 3: {1, 4}, 2: {4}, 4: {2, 3}}),
 ([4, 1], {2: {4}, 4: {1, 2}, 3: {1}, 1: {3, 4}}),
 ([4, 2], {1: {4}, 4: {1, 2}, 3: {2}, 2: {3, 4}}),
 ([4, 3], {1: {4}, 4: {1, 3}, 2: {3}, 3: {2, 4}}),
 ([4, 4], {1: {4}, 4: {1, 2, 3}, 2: {4}, 3: {4}})]

In [10]:
verify_results(n4_results)

'Passed!'